In [ ]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
_tmgTPB = mm.module('tmg.common.TMG_tool_page_builder')
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')


def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name[5:],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"


In [2]:
assignment_classes = {
                    "Express Bus": {
                        "demand_matrix": "mf1",
                        "actual_time_matrix": "mf2",
                        "perceived_time_matrix": "mf3",
                        "required_mode": set("gp"),
                        "other_modes": set('qn') | set('bsL'),
                        "aux_modes":  set('yvwzut'),
                        "vol_att": "@voltr_ex",
                        "must_use": True
                    }
                }

In [3]:
def get_journey_levels(name,parameters):

    #Level 0
    Level0 = {
        "boarding_cost": None,
        "boarding_time": None,
        "waiting_time": None,
    }
    Level0['description'] = 'init'
    Level0['destinations_reachable'] = False 
    transition_rules = []
    for mode in parameters["required_mode"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 1
        })
    for mode in parameters["other_modes"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 0
        })
    Level0["transition_rules"] = transition_rules

    #Level1
    Level1 = {
        "boarding_cost": None,
        "boarding_time": None,
        "waiting_time": None,
    }
    Level1['description'] = name
    Level1['destinations_reachable'] = True 
    transition_rules = []

    for mode in parameters["required_mode"]|parameters["other_modes"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 1
        })
    Level1["transition_rules"] = transition_rules

    return[Level0,Level1]

In [5]:
journey = get_journey_levels("Express Bus", assignment_classes["Express Bus"])

In [6]:
journey

[{'boarding_cost': None,
  'boarding_time': None,
  'description': 'init',
  'destinations_reachable': False,
  'transition_rules': [{'mode': 'p', 'next_journey_level': 1},
   {'mode': 'g', 'next_journey_level': 1},
   {'mode': 'q', 'next_journey_level': 0},
   {'mode': 's', 'next_journey_level': 0},
   {'mode': 'b', 'next_journey_level': 0},
   {'mode': 'L', 'next_journey_level': 0},
   {'mode': 'n', 'next_journey_level': 0}],
  'waiting_time': None},
 {'boarding_cost': None,
  'boarding_time': None,
  'description': 'Express Bus',
  'destinations_reachable': True,
  'transition_rules': [{'mode': 'q', 'next_journey_level': 1},
   {'mode': 'p', 'next_journey_level': 1},
   {'mode': 's', 'next_journey_level': 1},
   {'mode': 'b', 'next_journey_level': 1},
   {'mode': 'g', 'next_journey_level': 1},
   {'mode': 'L', 'next_journey_level': 1},
   {'mode': 'n', 'next_journey_level': 1}],
  'waiting_time': None}]

In [1]:
import inro.modeller as m
import os
import glob
import csv
import pandas as pd

mm = m.Modeller()
eb = m.Modeller().emmebank

#tools
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")           
change_matrix = mm.tool("inro.emme.data.matrix.change_matrix_properties")
create_matrix =  mm.tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = mm.tool("inro.emme.matrix_calculation.matrix_calculator")
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
network_calculator = mm.tool("inro.emme.network_calculation.network_calculator")
selectLine = mm.tool("inro.emme.transit_assignment.extended.strategy_based_analysis")

#HDR Scripts
#common = mm.module('org.emme.common')
#moe = mm.module('org.emme.moe')

In [2]:
matrix_list ={
    "trips_peak_transit_drive_to_and_from" :"",
    "trips_peak_transit_express_bus" :"",
    "trips_peak_transit_local_bus" :"",
    "trips_peak_transit_rapid_bus" :"",
    "trips_peak_transit_subway" :"",
    "trips_peak_transit_train" :"",
    "trips_peak_transit_ztsa" :"",
    "trips_peak_hov2_toll_nh" :"",
    "trips_peak_hov3" :"",
    "trips_peak_hov3_nt_hov" :"",
    "trips_peak_hov3_nt_nh" :"",
    "trips_peak_hov3_toll_hov" :"",
    "trips_peak_hov3_toll_nh" :"",
    }

In [3]:
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name,
                  matrix_description = matrix_name)

In [2]:
from copy import deepcopy

In [3]:
networkCalcSpec = {
    "result": "@sel_go",
    "expression": "0",
    "aggregation": None,
    "selections": {
        "link": "all",
        "transit_line": "all"
    },
    "type": "NETWORK_CALCULATION"
}


In [4]:
selectLink = {
    "trip_components": {
        "in_vehicle": "@sel_go",
        "aux_transit": None,
        "boarding": None,
        "alighting": None,
    },
    "sub_path_combination_operator": ".max.",
    "sub_strategy_combination_operator": ".max.",
    "selected_demand_and_transit_volumes": {
        "sub_strategies_to_retain": "ALL",
        "selection_threshold": {
            "lower": 1,
            "upper": 1
        }
    },
    "analyzed_demand": None,
    "constraint": None,
    "results": {
        "strategy_values": None,
        "selected_demand": None,
        "transit_volumes": None,
        "aux_transit_volumes": None,
        "total_boardings": None,
        "total_alightings": None
    },
    "type": "EXTENDED_TRANSIT_STRATEGY_ANALYSIS"
}

In [7]:
line = "GB25"

In [8]:
newMat.id

NameError: name 'newMat' is not defined

In [5]:
lines = ["GB12","GB15","GB16","GB18","GB19","GB20","GB21","GB24","GB25","GB27","GB29","GB31",
       "GB32","GB33","GB34","GB36","GB37","GB38","GB40","GB45","GB46","GB47","GB48","GB51","GB52",
       "GB54","GB65","GB66","GB68","GB70","GB71","GB81","GB88","GB90","GB91",
       "GB92","GB93","GB96"]

In [6]:
lines = ["GB12","GB15","GB16","GB18","GB19","GB20","GB21","GB24","GB25","GB27","GB29","GB31",
       "GB32","GB33","GB34","GB36","GB37","GB38","GB40","GB45","GB46","GB47","GB48","GB51","GB52",
       "GB54","GB65","GB66","GB68","GB70","GB71","GB81","GB88","GB90","GB91",
       "GB92","GB93","GB96"]
for line in lines:
    network_calculator(networkCalcSpec) # Clean network
    newMat = create_matrix(matrix_id = "mf" + line[-2:],
                      matrix_name= line,
                      matrix_description = "OD Demand for " + line)
    addLine = deepcopy(networkCalcSpec)
    addLine["expression"] = "1"
    addLine["selections"]["transit_line"] = "line = " + line + "__"
    network_calculator(addLine)
    runSelectLine = deepcopy(selectLink)
    runSelectLine["results"]["selected_demand"] = newMat.id
    selectLine(runSelectLine)

Error: specification.type: Expecting the following constant: EXTENDED_TRANSIT_PATH_ANALYSIS